<a href="https://colab.research.google.com/github/9tea9/Cse465_AIvsFAKE_image_project/blob/main/CSE465.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()  # This will prompt you to upload kaggle.json


In [ ]:
git remote add origin https://github.com/9tea9/Cse465_AIvsFAKE_image_project.git
git branch -M main
git push -u origin main


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Ensure proper permissions


In [ ]:
!pip install kaggle


In [ ]:
!kaggle datasets download -d birdy654/cifake-real-and-ai-generated-synthetic-images  # Replace with actual dataset path

In [ ]:
!unzip cifake-real-and-ai-generated-synthetic-images.zip



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
!ls train
!ls test

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load the training data
train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(128, 128),  # Resize images to 128x128
    batch_size=32,
    class_mode='binary'      # Binary classification: real or fake
)

# Load the test data
test_generator = test_datagen.flow_from_directory(
    'test',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
print(train_generator.class_indices)
print(test_generator.class_indices)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(128, 128),
    batch_size=16,  # Smaller batch size
    class_mode='binary'
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=8
)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def plot_accuracy(history):

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    epochs = range(1, len(acc) + 1)

    # Plot accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(epochs, acc, 'bo-', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

plot_accuracy(history)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

def plot_confusion_matrix(model, test_generator):

    # Get true labels and predictions
    test_generator.reset()
    y_true = test_generator.classes
    y_pred = model.predict(test_generator)
    y_pred_classes = (y_pred > 0.5).astype(int)  # Convert probabilities to binary class labels

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    class_labels = list(test_generator.class_indices.keys())

    # Plot confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
    disp.plot(cmap='Blues', values_format='d')
    plt.title('Confusion Matrix')
    plt.show()

plot_confusion_matrix(model, test_generator)


In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
def plot_test_metrics(test_loss, test_accuracy):

    # Data for plotting
    metrics = ['Test Accuracy', 'Test Loss']
    values = [test_accuracy * 100, test_loss]

    # Create a bar plot
    plt.figure(figsize=(8, 6))
    sns.barplot(x=metrics, y=values, palette='viridis')

    # Annotate the bars with the values
    for i, v in enumerate(values):
        plt.text(i, v + 0.5, f"{v:.2f}", ha='center', fontsize=12)

    # Customize plot
    plt.title('Test Metrics', fontsize=16)
    plt.ylabel('Value', fontsize=14)
    plt.xlabel('Metric', fontsize=14)
    plt.ylim(0, max(values) + 10)
    plt.show()
plot_test_metrics(test_loss, test_accuracy)


In [ ]:
model.save('cifake_model.keras')


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

model = load_model('/content/cifake_model.keras')

def predict_image(image_path):

    img = load_img(image_path, target_size=(128, 128))  # Resize to match model input
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    prediction = model.predict(img_array)
    return "Real" if prediction[0][0] > 0.5 else "AI-Generated"

image_path = "/content/Real_Horse.jpg"
result = predict_image(image_path)
print(f"The image is predicted to be: {result}")
